# Custom Layers in Tensorflow 

See, Tensorflow is made of individual layers. As users, we can make our own custom layers or modify the individual variables in Tensorflow layers.

In [1]:
import tensorflow as tf
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False


In [2]:
# A simple layer 
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)

#defining the input models is not necessary 
# input dimension is inferred automatically but also can be 
# expressed manually for complex operations 

layer = tf.keras.layers.Dense(10, input_shape = (None, 5))



There is a documentation of pre existing layers. It includes Dense, conv3d. LSTM
 etc.

In [6]:
# to use a layer, simply call it 
layer(tf.zeros([5,5]))

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [7]:
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.5682236 ,  0.32178557,  0.5541344 , -0.08963978,  0.41886717,
         -0.43304485,  0.08269399, -0.03028601,  0.41649967,  0.49176174],
        [-0.20151764,  0.21329445,  0.359567  , -0.5806151 ,  0.47501558,
         -0.5004013 , -0.5768603 ,  0.28010684,  0.41927415, -0.37717733],
        [ 0.38147563, -0.11047733,  0.13210171, -0.41349807,  0.21033561,
          0.62211937, -0.5997275 , -0.4022569 ,  0.04846305, -0.4869731 ],
        [ 0.34813327,  0.4363125 ,  0.4135993 , -0.357536  , -0.20942125,
          0.40269107, -0.01195818, -0.12818027,  0.60661346,  0.4269783 ],
        [-0.5849191 , -0.33763936, -0.6214134 ,  0.23072171,  0.25312775,
          0.16386425, -0.17402351, -0.02721953,  0.2582109 , -0.62539303]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [9]:
# the variables are also accessible through accessors 

layer.bias

<tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

## Now the fun , implementing custom layers 

The best way to implement your own layer is extending the tf.keras.Layer class and implementing:

__init__ , where you can do all input-independent initialization
build, where you know the shapes of the input tensors and can do the rest of the initialization
call, where you do the forward computation


In [10]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape = [int(input_shape[-1]),
                                           self.num_outputs])
    
  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)

In [13]:
_= layer(tf.zeros([10,5])) # Calling the layer 'builds' it

In [14]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer/kernel:0']


## Models : Composing Layers 

Many interesting layer-like things in machine learning models are implemented by composing existing layers. For example, each residual block in a resnet is a composition of convolutions, batch normalizations, and a shortcut. Layers can be nested inside other layers.



In [20]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [24]:
_ = block(tf.zeros([1, 2, 3, 3]))

ValueError: ignored

In [25]:
block.layers

In [26]:
len(block.variables)

18

In [27]:
block.summary()

Model: "resnet_identity_block_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            multiple                  5         
_________________________________________________________________
batch_normalization_6 (Batch multiple                  4         
_________________________________________________________________
conv2d_7 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_7 (Batch multiple                  8         
_________________________________________________________________
conv2d_8 (Conv2D)            multiple                  9         
_________________________________________________________________
batch_normalization_8 (Batch multiple                  12        
Total params: 42
Trainable params: 30
Non-trainable params: 12
______________________________________________

models which compose many layers simply call one layer after the other. This can be done in very little code using tf.keras.Sequential:

In [28]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1,(1,1),
                                                     input_shape = (
                                                         None, None, 3
                                                     )),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(2,1, padding = 'same'),
                              tf.keras.layers.BatchNormalization(),
                              tf.keras.layers.Conv2D(3, (1,1)),
                              tf.keras.layers.BatchNormalization()])

my_seq(tf.zeros([1,2,3,3]))

<tf.Tensor: shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [29]:
my_seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, None, None, 1)     4         
_________________________________________________________________
batch_normalization_9 (Batch (None, None, None, 1)     4         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, None, None, 2)     4         
_________________________________________________________________
batch_normalization_10 (Batc (None, None, None, 2)     8         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, None, None, 3)     9         
_________________________________________________________________
batch_normalization_11 (Batc (None, None, None, 3)     12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
___________________________________________________________